#FICO Analytic Challenge Week 8
##© Fair Isaac 2024

#Advanced Feature Generation
Feature generation in neural networks for fraud detection involves creating sophisticated features to better identify fraudulent activities. By transforming raw data into meaningful inputs, we improve the model's ability to detect subtle patterns of fraud, enhancing its predictive accuracy and robustness.

Purpose of Feature Generation:
- Enhance detection by highlighting behaviors associated with fraud.
- Improve accuracy by providing the best possible data representations.

Types of Features for Fraud Detection:
- Temporal Features: Capture transaction timing and frequency to spot unusual patterns.
- Relational Features: Analyze relationships between transaction entities to uncover discrepancies.

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive


In [ ]:
# Defining projects path and directory locations
path = '/content/drive/MyDrive/FICO Analytic Challenge/'
sys.path.append(path +'Data')
sys.path.append(path +'Week 04')
os.chdir(path)
print(os.getcwd())

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Folder's name that's holding dataset
data = 'Data'

In [ ]:
# dataset file prefix
trainFile = ['train']
testFile = ['test_A']
# testFile = ['test_B']
# testFile = ['test_C_notags']

# CSV filename and where features dataset will be read 
featureTrainFileSuffix="_advanced_features.csv"
featureTestFileSuffix="_advanced_features.csv"

filePathTrain=os.path.join(path + data, trainFile[0] + featureTrainFileSuffix)
filePathTest=os.path.join(path + data, testFile[0] + featureTestFileSuffix)

if not os.path.isfile(filePathTrain):
    featureTrainFileSuffix="_features.csv"
    filePathTrain=os.path.join(path + data, trainFile[0] + featureTrainFileSuffix)

if not os.path.isfile(filePathTest):
    featureTestFileSuffix="_features.csv"
    filePathTest=os.path.join(path + data, testFile[0] + featureTestFileSuffix)

# CSV filename and where features dataset will be saved
featureTrainFileSaveSuffix="_advanced_features.csv"
featureTestFileSaveSuffix="_advanced_features.csv"

trainsaveCSV = os.path.join(path + data, trainFile[0] + featureTrainFileSaveSuffix)
testsaveCSV = os.path.join(path + data, testFile[0] + featureTestFileSaveSuffix)

In [ ]:
def import_df(filename):
    df1 = pd.read_csv(filename)
    df1.reset_index(drop=True, inplace=True)
    df1['datetime'] =  pd.to_datetime(df1['transactionDateTime'])
    df1['transactionDateTime'] = pd.to_datetime(df1['transactionDateTime'])
    df1 = df1.sort_values(by=['pan','transactionDateTime'])
    return df1

In [ ]:
# Upload pre-processed data
#df1 = pd.read_csv('/content/drive/MyDrive/FICO Analytic Challenge/Data/train_advanced_features.csv')
#df = pd.read_csv('/content/drive/MyDrive/FICO Analytic Challenge/Data/test_C_notags_advanced_features.csv')
#df = pd.read_csv('/content/drive/MyDrive/FICO Analytic Challenge/Data/test_C_notags_features.csv')
# df = pd.read_csv('/content/drive/MyDrive/FICO Analytic Challenge/Data/train_advanced_features.csv')
#df = pd.concat([df1, df2])
# df = import_df(filePathTrain)
# df = import_df(filePathTest)


df1 = import_df(filePathTrain)
df2 = import_df(filePathTest)
df = pd.concat([df1, df2])

base_cols = ['pan', 'merchant', 'category', 'transactionAmount', 'first', 'last',
       'mdlIsFraudTrx', 'mdlIsFraudAcct', 'is_train', 'cardholderCountry',
       'cardholderState', 'transactionDateTime', 'gender', 'street', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 
       'merch_lat', 'merch_long', 'merchCountry', 'merchState', 'deltaTime']

In [ ]:
# # ONLY FOR TEST_C_NOTAGS
# df = import_df(filePathTest)

# # ONLY FOR TEST_C_NOTAGS
# base_cols = ['pan', 'merchant', 'category', 'transactionAmount', 'first', 'last', 'is_train', 'cardholderCountry',
#        'cardholderState', 'transactionDateTime', 'gender', 'street', 'zip',
#        'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 
#        'merch_lat', 'merch_long', 'merchCountry', 'merchState', 'deltaTime']

In [ ]:
# Look at the column names
df.columns

Index(['level_0', 'pan', 'merchant', 'category', 'transactionAmount', 'lat',
       'long', 'trans_num', 'unix_time', 'merch_lat', 'merch_long',
       'mdlIsFraudTrx', 'mdlIsFraudAcct', 'transactionDateTime', 'is_train',
       'merchCountry', 'merchState', 'cardholderCountry', 'cardholderState',
       'transactionHour', 'time_since_last_transaction', 'IsHighValue',
       'is_international', 'is_cnp', 'is_shoppingnet', 'is_grocery_net',
       'is_travel', 'IS_0_TO_5AM', 'average_spending', 'wednesday_buy',
       'monday_buy', 'spending_below_avg_20', 'spending_above_threshold',
       'outside_state_purchase', 'is_late_night', 'user_avg_amount',
       'RelativeAmount', 'deltaTime', 'amt_trend_5e', 'amt_trend_24h',
       'count_trend_1h', 'category_ratio', 'repeat_amt', 'index',
       'rolling_mean_14D', 'rolling_mean_60D', 'ratio_14D_to_60D', 'ewm_1D',
       '1m', 'repeat_hi_amt_1H', 'HighValue_International',
       'transactionHour_Risk', 'is_grocery_pos', 'outside_country',

# Advanced Feature Generation for Credit Card Fraud Detection

Though we have already covered somewhat advanced features in week 4, there is still information to be squeezed from the data. We will investigate 3 key concepts in this notebook:
* ratio variables
* exponential weighting
* repeat events



## Ratio Variables

Ratio variables are perfect for detecting anomalies in spending patterns over time. The problem with simple "last hour" or "last day" variables is that you have no knowledge about this customer's habits outside of that specified timeframe. It's possible that the user naturally fluctuates rapidly throughout the day or week, but we're lacking the necessary long-period information. Typically you want the numerator to be something which encapsulates a time over which periodic behaviors can undergo a full cycle. For example: a user's spending habits might correlate with their pay cycle (2 weeks), or paying off their bills such as a car payment or rent (1 month). We don't want these non-fraud behaviors to trigger any alarm in our variables, so we make the numerator long enough for it to average out. On the other hand, we want our denominator to capture several of these cycles to ensure the numerator truly is reflective of that user's spending patterns. Please create at least three ratio variables which capture transaction frequency or spending amount averages. You may want to use rolling windows for these.


In [ ]:
# Example: rolling mean of last 14 days (2 weeks) vs. rolling mean of last 60 days (2 months)
df.set_index('datetime', inplace=True)  # Set the date column as the index
df['rolling_mean_14D'] = df.groupby('pan')['transactionAmount'].rolling(window='14D', min_periods=1).mean().reset_index(level=0, drop=True)
df['rolling_mean_60D'] = df.groupby('pan')['transactionAmount'].rolling(window='60D', min_periods=1).mean().reset_index(level=0, drop=True)
df['ratio_14D_to_60D'] = df['rolling_mean_14D'] / df['rolling_mean_60D']
df.reset_index(drop=True, inplace=True)

If you want to alter the range of the window, simply change the window parameter in the rolling function. Here, we've specified '14D' and '60D' respectively. If you want to make a variable which goes over a window of a month, for instance, you could replace the parameter with '30D'

In [ ]:
# Making sure that NaN values are replaced with zeros because they can cause issues with our rolling average calculations
df = df.fillna(0)
df.reset_index(drop=True, inplace=True)

In [ ]:
# # Visualizing ratio variables that we have calulated
# pan_number = df['pan'].unique()[0]  # Select the first pan number for example
# subset = df[df['pan'] == pan_number]

# # Choose a time range (e.g., first 3 months)
# start_date = subset['datetime'].min()
# end_date = start_date + pd.DateOffset(months=3)
# subset = subset[(subset['datetime'] >= start_date) & (subset['datetime'] <= end_date)]

# plt.figure(figsize=(12, 6))
# plt.plot(subset['datetime'], subset['rolling_mean_14D'], label='Rolling Mean (7 days)')
# plt.plot(subset['datetime'], subset['transactionAmount'], label='Transaction Amount')
# plt.plot(subset['datetime'], subset['rolling_mean_60D'], label='Rolling Mean (30 days)')

# plt.title(f'Transaction Amount and Rolling Mean for PAN: {pan_number}')
# plt.xlabel('Date')
# plt.ylabel('Amount')
# plt.legend()
# plt.grid(True)
# plt.show()

Now, let's visualize the ratio of the 14-day rolling average and 60 day rolling average. Calculating this ratio can highlight periods where the short-term trend is significantly different from the long-term trend, indicating potential anomalies, seasonal effects, or sudden changes in behavior.

In [ ]:
# # Plotting ratio of 14-day rolling average to 60-day rolling average
# pan_number = df['pan'].unique()[0]  # Select the first pan number for example
# subset = df[df['pan'] == pan_number]

# # Choose a time range (e.g., first 3 months)
# start_date = subset['datetime'].min()
# end_date = start_date + pd.DateOffset(months=3)
# subset = subset[(subset['datetime'] >= start_date) & (subset['datetime'] <= end_date)]

# plt.figure(figsize=(12, 6))
# plt.plot(subset['datetime'], subset['ratio_14D_to_60D'], label='Rolling Ratio')

# plt.title(f'Transaction Amount and Rolling Mean for PAN: {pan_number}')
# plt.xlabel('Date')
# plt.ylabel('Ratio')
# plt.legend()
# plt.grid(True)
# plt.show()

## Exponential Weighting

We have some notion of how to leverage long-term spending patterns to detect fraud, but what if we wanted to detect the instantaneous frauds as they're occurring? Exponential event and time decay are designed specifically for this task. By giving priority to the most recent transaction, we are able to detect fraud even sometimes on the very first fraud attempt, saving banks even more money.


### Below is an example of the exponential weighted mean function over time

In [ ]:
df['datetime'] =  pd.to_datetime(df['transactionDateTime'])
# df.reset_index(drop=True,inplace=True)
# Define halflife in Days
halflife = 1.  #Day

df['ewm_1D'] = df.groupby('pan').apply(
    lambda group: group['transactionAmount'].ewm(halflife=halflife, adjust=False).mean()
).reset_index(level=0, drop=True)
# Making sure that NaN values are replaced with zeros because they can cause
# issues with our rolling average calculations
df = df.fillna(0)


In [ ]:
# # Plotting ratio of 14-day rolling average to 60-day rolling average
# pan_number = df['pan'].unique()[0]  # Select the first pan number for example
# subset = df[df['pan'] == pan_number]

# # Choose a time range (e.g., first 3 months)
# start_date = subset['datetime'].min()
# end_date = start_date + pd.DateOffset(months=3)
# subset = subset[(subset['datetime'] >= start_date) & (subset['datetime'] <= end_date)]

# plt.figure(figsize=(12, 6))
# plt.plot(subset['datetime'], subset['ewm_1D'], label='Rolling Ratio')
# plt.plot(subset['datetime'], subset['transactionAmount'], label='Transaction Amount')

# plt.title(f'Transaction Amount and Rolling Mean for PAN: {pan_number}')
# plt.xlabel('Date')
# plt.ylabel('Ratio')
# plt.legend()
# plt.grid(True)
# plt.show()

Here, halflife (the time it takes for the value to decay to half) is a parameter which dictates how much you would like to prioritize more recent events. The larger halflife is, the less you prioritize recent events, and the smaller halflife is, the more you emphasize recent events.

# Different profile filters distinguish different basic profile variable types
Examples:
- Time Averages
- Repeats

# Time Averages
- Weighted average rate of change over time
- Weighting function gives
    - Unit weight to current time
    - Exponential suppression for earlier events, e.g. e^[(𝑡_(𝑛−1)−𝑡_𝑛)/𝑇]

# Repeats
- Counts how many repeat events have a specified property
- Normal repeat counting goes as 1, 2, 3, 4, ….
- Falcon uses exponentially decayed repeat counting
- If current event qualifies as repeats, then multiply prior count by e^(−dt/T) before adding 1
- If current event does not qualify as repeat, then reset variable to 0


In [ ]:
def compute_timedecay_rate(df, decay=3600, name = "time_decay"):
    avg_amount_list = []
    current_count_rate = 0
    current_amount_rate = 0
    current_pan = None

    for index, row in df.iterrows():
        DC = np.exp(-row["deltaTime"]/decay)
        if (row["pan"] != current_pan):
            current_pan = row["pan"]
            current_count_rate = 0
            current_amount_rate = 0
            DC = 0
        current_count_rate = 1 + current_count_rate*DC
        current_amount_rate = row["transactionAmount"] + current_amount_rate*DC
        avg_amount = current_amount_rate / current_count_rate
        avg_amount_list.append(avg_amount)
    df[name] = avg_amount_list


In [ ]:
def compute_hi_repeat(df, decay=60, name = "1d"):

    avg_amount_list = []
    current_count_rate = 0
    current_amount_rate = 0
    current_pan = None

    df['condition'] = False
    df.loc[df['transactionAmount'] > 100., 'condition'] = True

    for index, row in df.iterrows():
        if ((row["pan"] != current_pan) or (row['condition'] == False)):
            current_pan = row["pan"]
            current_count_rate = 0
            current_amount_rate = 0
        current_count_rate = 1 + current_count_rate*np.exp(-row["deltaTime"]/decay)
        current_amount_rate = row["transactionAmount"] + current_amount_rate*np.exp(-row["deltaTime"]/decay)
        avg_amount = current_amount_rate / current_count_rate
        avg_amount_list.append(avg_amount)

    df[name] = avg_amount_list
    df.drop(columns='condition',inplace=True)


In [ ]:
import math
df['deltaTime'] = df.groupby('pan')['datetime'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()
compute_timedecay_rate(df, decay=60, name = "1m")
compute_hi_repeat(df, decay=3600, name = "repeat_hi_amt_1H")

In [ ]:
df[df['repeat_hi_amt_1H'] < df['transactionAmount']/3]['pan']

In [ ]:
# #pan_number = df[df['mdlIsFraudAcct'] == 1]['pan'].unique()[29]  # Select the first pan number for example
# pan_number = '2712209726293386A'
# subset = df[df['pan'] == pan_number]

# # Choose a time range (e.g., first 3 months)
# start_date = subset['datetime'].min()
# end_date = start_date + pd.DateOffset(months=3)
# subset = subset[(subset['datetime'] >= start_date) & (subset['datetime'] <= end_date)]

# plt.figure(figsize=(12, 6))
# plt.plot(subset['datetime'], subset['1m'], label='1m')
# plt.plot(subset['datetime'], subset['transactionAmount'], label='Transaction Amount')

# plt.title(f'1D for PAN: {pan_number}')
# plt.xlabel('Date')
# plt.ylabel('Amount')
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.figure(figsize=(12, 6))
# plt.plot(subset['datetime'], subset['repeat_hi_amt_1H'], label='repeat_hi_amt_1H')
# plt.plot(subset['datetime'], subset['transactionAmount'], label='Transaction Amount')

# plt.title(f'repeat_1D for PAN: {pan_number}')
# plt.xlabel('Date')
# plt.ylabel('Amount')
# plt.legend()
# plt.grid(True)
# plt.show()


## Now that we have a dataframe with some potentially useful features, let's save it to be used in future sessions.

In [ ]:
df.columns

In [ ]:
df.set_index('datetime', inplace=True)
df['AvgTransactionAmount_Last7Days'] = df.groupby('pan')['transactionAmount'].transform(lambda x: x.rolling(window='168h').mean())

In [ ]:
# df['datetime'] = pd.to_datetime(df['transactionDateTime'])

In [ ]:
df['HighValue_International'] = ((df['transactionAmount'] > 1000) & (df['is_international'] == 1)).astype(int)

In [ ]:
df['transactionHour_Risk'] = ((df['transactionHour'].astype(int) < 4) | (df['transactionHour'].astype(int) > 21)).astype(int)

In [ ]:
df.reset_index(drop=True, inplace=True)
df.set_index('datetime', inplace=True)
df['count_trend_1h'] = df.groupby('pan')['transactionAmount'].transform(lambda x: x.rolling(window='1h').count())

In [ ]:
def calculate_category_ratio(df):

    # Define a function to compute rolling category ratios for each customer
    def compute_ratios(group):
        # Create a boolean mask of matches within the past 5 records (rolling window)
        match_mask = (group['category'] == group['category'].shift(1))
        # Rolling sum of matches for past 5 rows
        rolling_sum = match_mask.rolling(window=5, min_periods=1).sum()
        # Calculate the ratio
        return rolling_sum / rolling_sum.rolling(window=5, min_periods=1).count()

    # Apply the function group-wise (for each customer)
    df['category_ratio'] = df.groupby('pan', group_keys=False).apply(compute_ratios)

    return df
df = calculate_category_ratio(df)

In [ ]:
# #df['weekend_weekday_ratio'] = df.groupby('pan').apply(lambda x: x[x['datetime'].dt.dayofweek.isin([5, 6])]['transactionAmount'].sum() / x[~x['datetime'].dt.dayofweek.isin([5, 6])]
#  ['transactionAmount'].sum()).shift().reset_index(level=0, drop=True)

In [ ]:
#df['is_shoppingnet'] = (df['category'].apply(lambda x: x == 'shopping_net')).astype(int)

In [ ]:
df['is_grocery_pos'] =(df['category'].apply(lambda x: x == 'grocery_pos')).astype(int)

In [ ]:
df['is_travel'] = (df['category'] == 'travel') & (df['transactionAmount'] > 400).astype(int)


In [ ]:
df['average_spending'] = df.groupby('pan')['transactionAmount'].transform('mean')
# This line is added to calculate and create the 'average_spending' column

In [ ]:
# df['weekend_weekday_ratio'] = df.groupby('pan').apply(lambda x: x[x['datetime'].dt.dayofweek.isin([5, 6])]['transactionAmount'].sum() / x[~x['datetime'].dt.dayofweek.isin([5, 6])]
#  ['transactionAmount'].sum()).shift().reset_index(level=0, drop=True)

In [ ]:
df.reset_index(drop=True, inplace=True)
df['datetime'] = pd.to_datetime(df['transactionDateTime'])

In [ ]:
df['monday_buy']=((df['datetime'].dt.dayofweek.isin([1]))).astype(int)

In [ ]:
df['spending_below_avg_20'] = (df['average_spending'] - df['transactionAmount'] >= 20).astype(int)

In [ ]:
df['spending_above_threshold'] = (df['transactionAmount'] > df['average_spending'] + 15).astype(int)

In [ ]:
df['outside_state_purchase'] = (df['merchState'] != df['cardholderState']).astype(int)

In [ ]:
df['outside_country']= (df['merchCountry']!= df['cardholderCountry']).astype(int)

In [ ]:
df['user_avg_amount'] = df.groupby('pan')['transactionAmount'].expanding().mean().shift().reset_index(level=0, drop=True)

#df['user_avg_amount'] = df.groupby('pan')['transactionAmount'].expanding().mean().shift().reset_index(level=0, drop=True)

In [ ]:
df['high_interaction'] = ((df['transactionAmount'] * df['user_avg_amount']) > df['user_avg_amount'].mean()).astype(int)

In [ ]:
# prompt: Create the 'ratio_30D_to_60D' that uses the last 60 days

df.set_index('datetime', inplace=True)

df['rolling_mean_30D'] = df.groupby('pan')['transactionAmount'].rolling(window='30D', min_periods=1).mean().reset_index(level=0, drop=True)
df['rolling_mean_60D'] = df.groupby('pan')['transactionAmount'].rolling(window='60D', min_periods=1).mean().reset_index(level=0, drop=True)
df['ratio_30D_to_60D'] = df['rolling_mean_30D'] / df['rolling_mean_60D']

In [ ]:
# Features to save
if "datetime" in df.columns:
  features = list(set(df.columns) - set(base_cols + ["datetime"]))
else:
  features = list(set(df.columns) - set(base_cols))

features.sort()
saveFeatures = [*base_cols, *features]
print(f"Features to save: {saveFeatures}")

In [ ]:
#df[df['is_train'] == 1].to_csv(f'/content/drive/MyDrive/FICO Analytic Challenge/Data/train_advanced_features.csv',index=False)
# df[df['is_train'] == 0].to_csv(f'/content/drive/MyDrive/FICO Analytic Challenge/Data/test_C_notags_advanced_features.csv',index=False)

df[df['is_train'] == 1][saveFeatures].to_csv(trainsaveCSV ,index=False)
df[df['is_train'] == 0][saveFeatures].to_csv(testsaveCSV,index=False)


In [ ]:
# # CHALLENGE: rolling mean of last 30 days (1 month) vs. rolling mean of last 60 days (2 months)
# df['rolling_mean_30D'] = #TODO
# df['ratio_30D_to_60D'] = #TODO

In [ ]:
# # CHALLENGE: Exponential weighted mean over time with a half-life of 4 days

# df['datetime'] =  pd.to_datetime(df['transactionDateTime'])
# df.reset_index(drop=True,inplace=True)
# # Define halflife in Days
# halflife = #TODO

# df['ewm_1D'] = df.groupby('pan').apply(
#     lambda group: group['transactionAmount'].ewm(halflife=halflife, adjust=False).mean()
# ).reset_index(level=0, drop=True)
# # Making sure that NaN values are replaced with zeros because they can cause
# # issues with our rolling average calculations
# df = df.fillna(0)
